In [1]:
# Part 1: Initialization and Loading PGN Files

# Import required libraries
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])

# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games


Initializing Stockfish engine...
Initializing Maia engines...


<UciProtocol (pid=18668)>: stderr >>        _
<UciProtocol (pid=18668)>: stderr >> |   _ | |
<UciProtocol (pid=18668)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18669)>: stderr >>        _
<UciProtocol (pid=18669)>: stderr >> |   _ | |
<UciProtocol (pid=18669)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18670)>: stderr >>        _
<UciProtocol (pid=18670)>: stderr >> |   _ | |
<UciProtocol (pid=18670)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18672)>: stderr >>        _
<UciProtocol (pid=18672)>: stderr >> |   _ | |
<UciProtocol (pid=18672)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18673)>: stderr >>        _
<UciProtocol (pid=18673)>: stderr >> |   _ | |
<UciProtocol (pid=18673)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=18674)>: stderr >>        _
<UciProtocol (pid=18674)>: stderr >> |   _ | |
<UciProtocol (pid=18674)>: 

In [2]:
# Part 2: Finding Classic Puzzles

# Function to find classic puzzles in a game
def find_classic_puzzles(game, stockfish_engine, threshold=2.00):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()
        player = board.turn  # True for White, False for Black

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        if len(info) < 2:
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000) / 100.0
        second_best_eval = info[1]["score"].relative.score(mate_score=10000) / 100.0

        # Check if the difference is large enough and not a simple recapture
        if abs(best_eval - second_best_eval) >= threshold and not board.is_capture(best_move):
            puzzles.append((fen_before, best_move, best_eval, second_best_eval))
            print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")

        board.push(move)

    return puzzles


In [3]:
# Part 3: Evaluating Maia Easiness

# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines):
    board = chess.Board(fen)
    easiness_scores = {}

    for elo, engine in maia_engines.items():
        result = engine.analyse(board, chess.engine.Limit(nodes=1))
        if "pv" in result:
            if best_move in result["pv"]:
                easiness_scores[elo] = result["pv"].index(best_move) / len(result["pv"])
            else:
                easiness_scores[elo] = None  # Best move not in the top suggestions
        else:
            easiness_scores[elo] = None  # No principal variation found

    return easiness_scores


In [4]:
# Part 4: Saving Results

# Function to save results incrementally to CSV
def save_results_to_csv(csv_file_path, data):
    with open(csv_file_path, "a", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for fen, best_move, best_eval, second_best_eval, maia_easiness in data:
            maia_easiness_str = {elo: score if score is not None else 'N/A' for elo, score in maia_easiness.items()}
            csvwriter.writerow([
                fen,
                best_move.uci(),  # Convert Move object to UCI notation
                best_eval,
                second_best_eval,
                json.dumps(maia_easiness_str)  # Store as JSON string for readability
            ])

# Function to save results incrementally to JSON
def save_results_to_json(json_file_path, data):
    if os.path.exists(json_file_path):
        with open(json_file_path, "r") as jsonfile:
            json_data = json.load(jsonfile)
    else:
        json_data = []

    for fen, best_move, best_eval, second_best_eval, maia_easiness in data:
        maia_easiness_str = {elo: score if score is not None else 'N/A' for elo, score in maia_easiness.items()}
        json_data.append({
            "fen": fen,
            "best_move": best_move.uci(),  # Convert Move object to UCI notation
            "best_eval": best_eval,
            "second_best_eval": second_best_eval,
            "maia_easiness": maia_easiness_str
        })

    with open(json_file_path, "w") as jsonfile:
        json.dump(json_data, jsonfile, indent=4)


In [5]:
# Part 5: Running the Analysis

# Paths to save results
csv_file_path = "classic_puzzles.csv"
json_file_path = "classic_puzzles.json"

# Initialize CSV file with headers
with open(csv_file_path, "w", newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["FEN", "Best Move", "Best Eval", "Second Best Eval", "Maia Easiness"])

# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")

for pgn_file in os.listdir(pgn_directory):
    if pgn_file.endswith(".pgn"):
        pgn_file_path = os.path.join(pgn_directory, pgn_file)
        print(f"Analyzing {pgn_file_path}...")
        all_games = load_games_from_pgn(pgn_file_path)

        for game in all_games:
            print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
            puzzles = find_classic_puzzles(game, stockfish_engine)
            for fen, best_move, best_eval, second_best_eval in puzzles:
                maia_easiness = evaluate_maia_easiness(fen, best_move, maia_engines)
                all_puzzles.append((fen, best_move, best_eval, second_best_eval, maia_easiness))

            # Save results incrementally
            save_results_to_csv(csv_file_path, all_puzzles)
            save_results_to_json(json_file_path, all_puzzles)

print("Classic puzzles analysis complete. Results saved.")

# Cleanup Engines
print("Cleaning up engines...")
stockfish_engine.quit()
for maia_engine in maia_engines.values():
    maia_engine.quit()
print("Engines cleaned.")


Starting to find classic puzzles in PGN files...
Analyzing analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn...
Loaded 4 games from PGN file analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn.
Analyzing game FIDE World Championship Match 1969
Puzzle found: 8/6kp/2Pn2p1/3B4/1p1R2P1/7P/2rp4/6K1 w - - 0 40, Best move: d5b3, Eval difference: 2.18
Puzzle found: 8/5B1R/5kp1/2n4P/8/5K2/1pr5/8 w - - 0 50, Best move: f7a2, Eval difference: 2.3099999999999996
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Puzzle found: 4r1k1/pp3pp1/7p/P1Rp2q1/3P1nP1/1P1bNP2/3Q1K1P/5B2 b - - 4 40, Best move: g5h4, Eval difference: 2.25
Analyzing analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn...
Loaded 5 games from PGN file analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn.
Analyzing game FIDE World Championship Match 1986
Analyzing game FIDE World Championship 